**Pair Programming ETL Transformación I**

In [6]:
from IPython.core.interactiveshell import InteractiveShell # Nos permite mostar más de una salida por celda
InteractiveShell.ast_node_interactivity = "all" # Nos permite mostar más de una salida por celda

import requests
import pandas as pd
import numpy as np
import ast
pd.options.display.max_rows=None
pd.options.display.max_columns=None

In [3]:
clima=pd.read_csv('datos/df_ETL1.csv', index_col = 0)
clima.reset_index(inplace = True, drop = True)

clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


In [5]:
df_attacks= pd.read_csv ('datos/df_attacks_limpieza_completa.csv', index_col=0)

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.

En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD.

En este momento tenemos dos fuentes de datos:


1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.


El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes.

Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:

USA

Australia

New Zealand

South Africa

Papua New Guinea

Del dataframe de los datos climáticos seleccionaremos todas las columnas.

Cuando ya tengamos todos los datos deseados juntaremos los dos csv.

Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.

Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

--------------------


Seleccionamos solo los paises que nos interesan.

In [7]:
df_attacks2 = df_attacks[df_attacks['country'].isin (['usa','australia', 'papua new guinea','new zealand','south africa'])]
df_attacks2.head(2)


,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860


In [8]:
clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


In [9]:
clima['wind_profile'].dtype

dtype('O')

Casteamos las dos columnas con diccionarios para que las convierta al tipo correspondiente para poder hacer apply del metodo pd.Series

In [10]:
clima['wind_profile']=clima['wind_profile'].apply(ast.literal_eval)

In [11]:
clima['rh_profile']= clima['rh_profile'].apply(ast.literal_eval)

In [12]:
y_rh = clima['rh_profile'].apply(pd.Series) 
y_rh.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 9}","{'layer': '850mb', 'rh': 6}","{'layer': '800mb', 'rh': 5}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 2}","{'layer': '650mb', 'rh': 0}","{'layer': '600mb', 'rh': 0}","{'layer': '550mb', 'rh': 1}","{'layer': '500mb', 'rh': 13}","{'layer': '450mb', 'rh': 16}","{'layer': '400mb', 'rh': 16}","{'layer': '350mb', 'rh': 16}","{'layer': '300mb', 'rh': 13}","{'layer': '250mb', 'rh': -2}","{'layer': '200mb', 'rh': -3}"
1,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 10}","{'layer': '850mb', 'rh': 7}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 4}","{'layer': '700mb', 'rh': 1}","{'layer': '650mb', 'rh': -1}","{'layer': '600mb', 'rh': 1}","{'layer': '550mb', 'rh': 5}","{'layer': '500mb', 'rh': 10}","{'layer': '450mb', 'rh': 9}","{'layer': '400mb', 'rh': 11}","{'layer': '350mb', 'rh': 12}","{'layer': '300mb', 'rh': 1}","{'layer': '250mb', 'rh': 0}","{'layer': '200mb', 'rh': -3}"
2,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 8}","{'layer': '850mb', 'rh': 11}","{'layer': '800mb', 'rh': 6}","{'layer': '750mb', 'rh': 3}","{'layer': '700mb', 'rh': 1}","{'layer': '650mb', 'rh': -1}","{'layer': '600mb', 'rh': 1}","{'layer': '550mb', 'rh': 2}","{'layer': '500mb', 'rh': -2}","{'layer': '450mb', 'rh': -3}","{'layer': '400mb', 'rh': -3}","{'layer': '350mb', 'rh': 1}","{'layer': '300mb', 'rh': 4}","{'layer': '250mb', 'rh': 2}","{'layer': '200mb', 'rh': -2}"
3,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 9}","{'layer': '850mb', 'rh': 8}","{'layer': '800mb', 'rh': 4}","{'layer': '750mb', 'rh': 1}","{'layer': '700mb', 'rh': -1}","{'layer': '650mb', 'rh': -2}","{'layer': '600mb', 'rh': 1}","{'layer': '550mb', 'rh': -2}","{'layer': '500mb', 'rh': -2}","{'layer': '450mb', 'rh': -2}","{'layer': '400mb', 'rh': 4}","{'layer': '350mb', 'rh': 9}","{'layer': '300mb', 'rh': 16}","{'layer': '250mb', 'rh': 14}","{'layer': '200mb', 'rh': 2}"
4,"{'layer': '950mb', 'rh': -9999}","{'layer': '900mb', 'rh': 11}","{'layer': '850mb', 'rh': 10}","{'layer': '800mb', 'rh': 8}","{'layer': '750mb', 'rh': 4}","{'layer': '700mb', 'rh': 0}","{'layer': '650mb', 'rh': 0}","{'layer': '600mb', 'rh': -2}","{'layer': '550mb', 'rh': -3}","{'layer': '500mb', 'rh': -1}","{'layer': '450mb', 'rh': 2}","{'layer': '400mb', 'rh': 6}","{'layer': '350mb', 'rh': 0}","{'layer': '300mb', 'rh': 3}","{'layer': '250mb', 'rh': 6}","{'layer': '200mb', 'rh': 5}"


Hacemos un for loop para sacar el df correspondiente a las columnas rh

In [13]:

for i in range(len(y_rh.columns)): 

    nombre = "rh_" + str(y_rh[i].apply(pd.Series)["layer"][0]) 
  
    
    valores = list(y_rh[i].apply(pd.Series)["rh"] )
    
    clima.insert(i, nombre, valores)

In [14]:
clima.head(2)

,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,-9999,9,6,5,3,2,0,0,1,13,16,16,16,13,-2,-3,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,-9999,10,7,6,4,1,-1,1,5,10,9,11,12,1,0,-3,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


Y repetimos el for loop para las columnas del wind, lo hacemos dos veces una para sacar los valores correspondientes a las columnas de direction y otra de speed.

In [15]:
x_wind = clima['wind_profile'].apply(pd.Series)
x_wind.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 350, 'speed': 5}","{'layer': '850mb', 'direction': 350, 'speed': 5}","{'layer': '800mb', 'direction': 345, 'speed': 5}","{'layer': '750mb', 'direction': 335, 'speed': 5}","{'layer': '700mb', 'direction': 335, 'speed': 6}","{'layer': '650mb', 'direction': 340, 'speed': 5}","{'layer': '600mb', 'direction': 340, 'speed': 5}","{'layer': '550mb', 'direction': 335, 'speed': 5}","{'layer': '500mb', 'direction': 315, 'speed': 4}","{'layer': '450mb', 'direction': 295, 'speed': 5}","{'layer': '400mb', 'direction': 300, 'speed': 5}","{'layer': '350mb', 'direction': 295, 'speed': 5}","{'layer': '300mb', 'direction': 290, 'speed': 5}","{'layer': '250mb', 'direction': 325, 'speed': 6}","{'layer': '200mb', 'direction': 320, 'speed': 8}"
1,"{'layer': '950mb', 'direction': -9999, 'speed'...","{'layer': '900mb', 'direction': 340, 'speed': 3}","{'layer': '850mb', 'direction': 345, 'speed': 5}","{'layer': '800mb', 'direction': 340, 'speed': 5}","{'layer': '750mb', 'direction': 335, 'speed': 5}","{'layer': '700mb', 'direction': 335, 'speed': 5}","{'layer': '650mb', 'direction': 340, 'speed': 5}","{'layer': '600mb', 'direction': 335, 'speed': 5}","{'layer': '550mb', 'direction': 325, 'speed': 5}","{'layer': '500mb', 'direction': 320, 'speed': 5}","{'layer': '450mb', 'direction': 320, 'speed': 5}","{'layer': '400mb', 'direction': 320, 'speed': 5}","{'layer': '350mb', 'direction': 325, 'speed': 5}","{'layer': '300mb', 'direction': 350, 'speed': 7}","{'layer': '250mb', 'direction': 335, 'speed': 8}","{'layer': '200mb', 'direction': 325, 'speed': 8}"


In [16]:

for i in range(len(x_wind.columns)): 

    nombre = "direction" + str(x_wind[i].apply(pd.Series)["layer"][0]) 

    valores = list(x_wind[i].apply(pd.Series)["direction"] )

    clima.insert(i, nombre, valores)

In [17]:
clima.head(2)

,direction950mb,direction900mb,direction850mb,direction800mb,direction750mb,direction700mb,direction650mb,direction600mb,direction550mb,direction500mb,direction450mb,direction400mb,direction350mb,direction300mb,direction250mb,direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,-9999,350,350,345,335,335,340,340,335,315,295,300,295,290,325,320,-9999,9,6,5,3,2,0,0,1,13,16,16,16,13,-2,-3,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,-9999,340,345,340,335,335,340,335,325,320,320,320,325,350,335,325,-9999,10,7,6,4,1,-1,1,5,10,9,11,12,1,0,-3,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


In [18]:
#este for loop es para el x_wind
for i in range(len(x_wind.columns)): 

    
    nombre = "speed" + str(x_wind[i].apply(pd.Series)["layer"][0]) 
   
    valores = list(x_wind[i].apply(pd.Series)["speed"] )

    clima.insert(i, nombre, valores)

Como en los tres for loops haciamos un insert, nuestro df clima ya tiene añadidas todas las filas resultantes. Para poder hacer un mergeo con el df attacks2 es necesario tener una columna en comun. Para ello unificamos los valores de la columna pais.

In [19]:
clima.head(2)

,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction950mb,direction900mb,direction850mb,direction800mb,direction750mb,direction700mb,direction650mb,direction600mb,direction550mb,direction500mb,direction450mb,direction400mb,direction350mb,direction300mb,direction250mb,direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,-9999,5,5,5,5,6,5,5,5,4,5,5,5,5,6,8,-9999,350,350,345,335,335,340,340,335,315,295,300,295,290,325,320,-9999,9,6,5,3,2,0,0,1,13,16,16,16,13,-2,-3,3,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",-2,15,12,1032,none,0,-9999,300,3,USA
1,-9999,3,5,5,5,5,5,5,5,5,5,5,5,7,8,8,-9999,340,345,340,335,335,340,335,325,320,320,320,325,350,335,325,-9999,10,7,6,4,1,-1,1,5,10,9,11,12,1,0,-3,6,6,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': -9999}, {'layer': '9...","[{'layer': '950mb', 'direction': -9999, 'speed...",3,15,7,1031,none,0,1,330,3,USA


In [20]:
clima['pais'] = clima['pais'].apply (lambda x : x.lower() )

In [21]:
clima['pais'].unique()

array(['usa', 'australia', 'south africa', 'new zealand',
       'papua new guinea'], dtype=object)

Ahora agrupamos el df clima por paises y le pedimos la media  de las variables.

In [22]:
df_clima_agrupado = clima.groupby (['pais']).mean()

C:\Users\marta\AppData\Local\Temp\ipykernel_20892\3231644396.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clima_agrupado = clima.groupby (['pais']).mean()


Guardamos el df resultante por seguridad.

In [23]:
df_clima_agrupado.to_csv('datos/clima_agrupado.csv')

In [24]:
df_clima_agrupado.head(2)

,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction950mb,direction900mb,direction850mb,direction800mb,direction750mb,direction700mb,direction650mb,direction600mb,direction550mb,direction500mb,direction450mb,direction400mb,direction350mb,direction300mb,direction250mb,direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
pais,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
australia,3.968750,4.078125,3.750000,3.640625,3.609375,3.671875,3.890625,4.00000,3.4375,3.28125,3.109375,3.171875,4.109375,4.859375,5.59375,5.640625,100.62500,100.234375,103.750,112.03125,122.109375,137.578125,148.828125,153.43750,151.71875,157.187500,169.765625,200.156250,229.843750,251.093750,260.390625,263.515625,1.234375,2.203125,3.406250,4.953125,7.156250,10.109375,11.40625,9.359375,2.546875,0.296875,-0.1250,-1.078125,-0.578125,-0.3750,-1.453125,-2.40625,97.5,3.546875,-9999.0,-9999.0,-9999.0,33.56250,2.1875,0.75000,1006.703125,0.234375,0.0,102.578125,3.140625
new zealand,-2341.953125,2.343750,2.390625,2.515625,2.765625,2.656250,2.562500,2.71875,2.8750,3.09375,3.234375,3.625000,4.203125,4.765625,5.12500,5.000000,-2242.03125,168.750000,191.875,205.78125,197.109375,185.000000,178.437500,179.84375,200.46875,193.046875,188.281250,178.828125,185.390625,180.078125,166.484375,168.828125,-2336.093750,10.046875,10.671875,10.468750,8.890625,3.656250,0.84375,0.296875,0.921875,0.812500,1.5625,2.109375,3.390625,5.6875,6.406250,2.50000,97.5,5.187500,-9999.0,-9999.0,-9999.0,15.34375,5.2500,9.90625,1017.031250,1.828125,0.0,151.015625,2.031250


Juntamos este df_clima_agrupado con el de df_attacks.

In [25]:
df_juntos=pd.merge(df_attacks2, df_clima_agrupado, right_on='pais', left_on='country')

In [26]:
df_juntos.head(2)

,year,type,country,age,species_,fecha_limpia,fatal,sex,latitud,longitud,country2,fatal_N,fatal_Unknown,fatal_Y,fatal_N.1,fatal_Unknown.1,fatal_Y.1,species_.1,fecha_limpia.1,type.1,age_NORM,speed950mb,speed900mb,speed850mb,speed800mb,speed750mb,speed700mb,speed650mb,speed600mb,speed550mb,speed500mb,speed450mb,speed400mb,speed350mb,speed300mb,speed250mb,speed200mb,direction950mb,direction900mb,direction850mb,direction800mb,direction750mb,direction700mb,direction650mb,direction600mb,direction550mb,direction500mb,direction450mb,direction400mb,direction350mb,direction300mb,direction250mb,direction200mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,2018,Boating,usa,57.0,White shark,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,4,7,1,9.116327,-9999.0,-152.734375,4.015625,4.0625,4.0625,4.109375,4.265625,4.625,4.984375,5.328125,5.828125,6.359375,7.046875,7.78125,8.78125,9.296875,-9999.0,53.296875,229.453125,241.25,248.125,256.5625,261.953125,274.765625,273.125,271.796875,268.4375,266.484375,259.53125,264.921875,264.21875,262.65625,-9999.0,-148.375,6.8125,5.3125,5.46875,5.515625,5.75,6.25,6.8125,7.078125,7.4375,9.140625,10.25,10.0625,6.265625,2.515625,97.5,6.703125,-9999.0,-9999.0,-9999.0,2.28125,13.34375,9.28125,1009.59375,0.796875,-468.09375,210.46875,2.71875
1,2018,Unprovoked,usa,11.0,Unespecific,Jun,N,F,39.78373,-100.445882,HN,1,0,0,1,0,0,3,7,7,3.804860,-9999.0,-152.734375,4.015625,4.0625,4.0625,4.109375,4.265625,4.625,4.984375,5.328125,5.828125,6.359375,7.046875,7.78125,8.78125,9.296875,-9999.0,53.296875,229.453125,241.25,248.125,256.5625,261.953125,274.765625,273.125,271.796875,268.4375,266.484375,259.53125,264.921875,264.21875,262.65625,-9999.0,-148.375,6.8125,5.3125,5.46875,5.515625,5.75,6.25,6.8125,7.078125,7.4375,9.140625,10.25,10.0625,6.265625,2.515625,97.5,6.703125,-9999.0,-9999.0,-9999.0,2.28125,13.34375,9.28125,1009.59375,0.796875,-468.09375,210.46875,2.71875


Guardamos df_juntos.

In [27]:
df_juntos.to_csv('datos/df_junto.csv')